In [1]:
import jsonlines
from transformers import AutoTokenizer, LlamaForCausalLM, AutoConfig
import torch
from datasets import load_dataset
import datasets
import os
from tqdm.notebook import tqdm
import json

In [16]:
!mkdir train
!mkdir test

In [11]:
### Process training data

In [2]:
def process_data(example):
    context = f"Instruction: {example['instruction']}\nInput: {example['input']}\nAnswer: "
    target = example['output']
    return {'context': context, 'target': target}


def process_data_chat(example):
    context = f"Instruction: {example['instruction']}\nInput: {example['input']}\nAnswer: {example['output']}"
    target = example['output']
    return {"chat": [
                {'role': 'system', 'content': "You are a helpful assistant and expert in finance, you answer questions very concisely"},
                {'role': 'user', 'content': f"Instruction: {example['instruction']}\nInput: {example['input']}"},
                {'role': 'assistant', 'content': example['output']}
           ]}
        

def process_save_data(name, file_name, split="train"):
    all_data = datasets.load_dataset(name)[split]
    
    processed_data_normal = all_data.map(process_data)
    processed_data_normal = processed_data_normal.remove_columns(["input", "output", "instruction"])
    print("normal example:", processed_data_normal[0])
   
    with jsonlines.open(f"{split}/{file_name}.jsonl", 'w') as writer:
      writer.write_all(processed_data_normal)

    processed_data_chat = all_data.map(process_data_chat)
    processed_data_chat = processed_data_chat.remove_columns(["input", "output", "instruction"])
    print("chat format example:", processed_data_chat[0])
    with jsonlines.open(f"{split}/{file_name}_chat.jsonl", 'w') as writer:
      writer.write_all(processed_data_chat)


In [3]:
process_save_data("FinGPT/fingpt-sentiment-train", "fingpt_sentiment_train")
process_save_data("FinGPT/fingpt-ner-cls", "fingpt_ner_cls_train")
process_save_data("FinGPT/fingpt-headline", "fingpt_headline_train")

normal example: {'context': 'Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: Teollisuuden Voima Oyj , the Finnish utility known as TVO , said it shortlisted Mitsubishi Heavy s EU-APWR model along with reactors from Areva , Toshiba Corp. , GE Hitachi Nuclear Energy and Korea Hydro & Nuclear Power Co. .\nAnswer: ', 'target': 'neutral'}
chat format example: {'chat': [{'content': 'You are a helpful assistant and expert in finance, you answer questions very concisely', 'role': 'system'}, {'content': 'Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: Teollisuuden Voima Oyj , the Finnish utility known as TVO , said it shortlisted Mitsubishi Heavy s EU-APWR model along with reactors from Areva , Toshiba Corp. , GE Hitachi Nuclear Energy and Korea Hydro & Nuclear Power Co. .', 'role': 'user'}, {'content': 'neutral', 'role': 'assistant'}]}
normal example: {'context'

In [4]:
# process_save_data("FinGPT/fingpt-sentiment-train", "fingpt_sentiment_test", "test")
process_save_data("FinGPT/fingpt-ner-cls", "fingpt_ner_cls_test", "test")
process_save_data("FinGPT/fingpt-headline", "fingpt_headline_test", "test")

Map:   0%|          | 0/3502 [00:00<?, ? examples/s]

example: {'context': "Instruction: What is the entity type of 'EVERGREEN SOLAR' in the input sentence.\nOptions: location, organization, person\nInput: Subordinated Loan Agreement - Silicium de Provence SAS and Evergreen Solar Inc . 7 - December 2007 [ HERBERT SMITH LOGO ] ................................ 2007 SILICIUM DE PROVENCE SAS and EVERGREEN SOLAR , INC .\nAnswer: ", 'target': 'organization'}


Map:   0%|          | 0/20547 [00:00<?, ? examples/s]

example: {'context': "Instruction: Does the news headline talk about price? Please choose an answer from {Yes/No}.\nInput: Trading partners unlikely to take India's gold import curbs lightly\nAnswer: ", 'target': 'No'}
